In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

# train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)
# test_data, val_data = train_test_split(test_data, test_size=0.5, random_state=42)
df = pd.read_json('/root/datasets/patch_db.json')

label2id = {'non-security':0,'security':1}
df = df.replace({"category": label2id})

train, test = train_test_split(df, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.1, random_state=42)
# 1. 数据预处理和特征提取

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train['diff_code'])
y_train = train['category']

/tmp/ipykernel_3467/3247451427.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"category": label2id})


In [2]:
test

,CVE_ID,CWE_ID,category,commit_id,commit_message,diff_code,owner,repo,source
31595,NA,NA,1,3927c3aa28ea2ed8dbb5228de3e69928972da412,commit 3927c3aa28ea2ed8dbb5228de3e69928972da41...,diff --git a/numpy/core/src/multiarray/ctors.c...,numpy,numpy,wild
33462,CVE-2016-3713,284,1,9842df62004f366b9fed2423e24df10542ee0dc5,From 9842df62004f366b9fed2423e24df10542ee0dc5 ...,diff --git a/arch/x86/kvm/mtrr.c b/arch/x86/kv...,torvalds,linux,cve
29183,NA,NA,1,2475f1a83ccf313d828b25f1769e3a37442ecf64,commit 2475f1a83ccf313d828b25f1769e3a37442ecf6...,diff --git a/libavcodec/anm.c b/libavcodec/anm...,libav,libav,wild
20626,NA,NA,0,94296c9697988be3cd6c0211e287f0e6c00be37f,commit 94296c9697988be3cd6c0211e287f0e6c00be37...,diff --git a/mongoose.c b/mongoose.c\nindex 8b...,civetweb,civetweb,wild
24391,NA,NA,1,1ff1401ff1d4b1fa7e76928b16025dbc039a5d58,commit 1ff1401ff1d4b1fa7e76928b16025dbc039a5d5...,diff --git a/libopenjpeg/t2.c b/libopenjpeg/t2...,szukw000,openjpeg,wild
...,...,...,...,...,...,...,...,...,...
13573,NA,NA,0,9bbbca3a0ee09293108b67835c6bdf6196d7bcb3,commit 9bbbca3a0ee09293108b67835c6bdf6196d7bcb...,diff --git a/drivers/md/md.c b/drivers/md/md.c...,stoth68000,media-tree,wild
14943,NA,NA,0,3aa141879fd9fa48ec7d71b9398f680d2adf90f2,commit 3aa141879fd9fa48ec7d71b9398f680d2adf90f...,diff --git a/misc-utils/getopt.c b/misc-utils/...,karelzak,util-linux,wild
13144,NA,NA,0,9d3d70c613e22896617d1a86459f2b41060566fe,commit 9d3d70c613e22896617d1a86459f2b41060566f...,diff --git a/lib/common/ipc.c b/lib/common/ipc...,ClusterLabs,pacemaker,wild
2142,NA,NA,0,046f3e821f146d6c473edb033af15a0604957af8,commit 046f3e821f146d6c473edb033af15a0604957af...,diff --git a/drivers/s390/block/dasd_eer.c b/d...,mjg59,linux,wild


In [3]:
# 3. 训练决策树分类器
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# 4. 评估模型
y_pred = clf.predict(vectorizer.transform(test['diff_code']))
print("Accuracy:", accuracy_score(test['category'], y_pred))
print("Classification Report:\n", classification_report(test['category'], y_pred, digits=4))

Accuracy: 0.6911908418260505
Classification Report:
               precision    recall  f1-score   support

           0     0.7704    0.7649    0.7676      4777
           1     0.5360    0.5436    0.5397      2386

    accuracy                         0.6912      7163
   macro avg     0.6532    0.6543    0.6537      7163
weighted avg     0.6923    0.6912    0.6917      7163

